In [1]:
import tensorflow as tf
from tensorflow.keras import layers


In [4]:
"""
一般只用到前三个参数：
● inputs就是该层的输入；
● units为神经元数目，即经过该层后的输出维度，也就是会将inputs的最后一个维度变成units；
● activation选择你所用的激活函数。
"""

model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_accuracy])

In [6]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))

model.fit(train_x, train_y, epochs=1, batch_size=100,
          validation_data=(val_x, val_y))

10/10 [==============================] - 0s 10ms/step - loss: 77.9201 - categorical_accuracy: 0.1060 - val_loss: 85.3687 - val_categorical_accuracy: 0.1050


In [ ]:
##tf.data输入数据   tf.data.Dataset.from_tensor_slices  对第一维度进行切分  返回一个切片数据集

dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.batch(32)
dataset = dataset.repeat()
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(32)
val_dataset = val_dataset.repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset, validation_steps=3)

In [17]:
# 评估
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)
# predict
result = model.predict(test_x, batch_size=32)
# print(result)

32/32 [==============================] - 0s 645us/step


In [15]:
tf.data.Dataset.from_tensor_slices((test_x, test_y))

<_TensorSliceDataset element_spec=(TensorSpec(shape=(72,), dtype=tf.float64, name=None), TensorSpec(shape=(10,), dtype=tf.float64, name=None))>

In [23]:
input_x = tf.keras.Input(shape=(72,))
hidden1 = layers.Dense(32, activation='relu')(input_x)
hidden2 = layers.Dense(16, activation='relu')(hidden1)
pred = layers.Dense(10, activation='softmax')(hidden2)

model = tf.keras.Model(inputs=input_x, outputs=pred)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 12.4851 - accuracy: 0.1140
Epoch 2/5
32/32 [==============================] - 0s 817us/step - loss: 19.4861 - accuracy: 0.1070
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 44.6978 - accuracy: 0.0960
Epoch 4/5
32/32 [==============================] - 0s 829us/step - loss: 77.7174 - accuracy: 0.0920
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 127.7407 - accuracy: 0.0880


In [27]:
# 模型子类化
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.layer1 = layers.Dense(32, activation='relu')
        self.layer2 = layers.Dense(num_classes, activation='softmax')
    def call(self, inputs):
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

# model = MyModel(num_classes=10)
# model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
#              loss=tf.keras.losses.categorical_crossentropy,
#              metrics=['accuracy'])

# model.fit(train_x, train_y, batch_size=16, epochs=5)

In [28]:
model = MyModel(num_classes=10)
model(np.random.random((1000, 72)))

<tf.Tensor: shape=(1000, 10), dtype=float32, numpy=
array([[0.01993992, 0.04682544, 0.04629159, ..., 0.14242508, 0.0336147 ,
        0.11927836],
       [0.03565971, 0.03682049, 0.06283499, ..., 0.07878938, 0.01869933,
        0.09366627],
       [0.05375763, 0.03913834, 0.0991307 , ..., 0.05522916, 0.02985952,
        0.08152535],
       ...,
       [0.02118999, 0.04070491, 0.05563221, ..., 0.06464586, 0.03621652,
        0.03931456],
       [0.07808124, 0.04516015, 0.07458433, ..., 0.07993537, 0.03087524,
        0.09578998],
       [0.0254843 , 0.03435217, 0.07008049, ..., 0.06254385, 0.04078537,
        0.12039164]], dtype=float32)>